In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
from graph_utils import graph_gen
from algorithms import IPLUX

# Set up the logger to print info messages for understandability.
import logging
import sys
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='')

# from algorithms import B_DPP, Falsone, C_SP_SG, DPD_TV

num_node = 20
num_edge = 40

graph_gen(num_node, num_edge, B=1)

net_dir = f'data/graph/N{num_node}E{num_edge}'
network = np.load(f'{net_dir}/subgraph_W.npy')
print(network.shape)
# print(network)

Graph with 20 nodes and 40 edges


C:\Users\xtq\AppData\Roaming\Python\Python311\site-packages\networkx\drawing\nx_pylab.py:315: UserWarning: 

The arrowsize keyword argument is not applicable when drawing edges
with LineCollection.

To make this warning go away, either specify `arrows=True` to
force FancyArrowPatches or use the default values.
Note that using FancyArrowPatches may be slow for large graphs.

  draw_networkx_edges(G, pos, arrows=arrows, **edge_kwds)


(20, 20)


In [6]:
from problem import Synthetic

parameters = {}
parameters['N'] = num_node
parameters['d'] = 3
parameters['p'] = 1
parameters['m'] = 5

prob = Synthetic(parameters)
prob.gen()
prob.load()


generating a Synthetic problem: N=20, d=3, p=1, m=5
Q: (20, 3), P: (20, 3, 3)
A: (20, 5, 3)
a: (20, 3), c: (20,)
aa: (20, 3), ca: (20,)
x* [[-5.68671446e-01  2.81746424e-11  1.77616960e-09]
 [-1.72560090e-10 -1.34023187e-10  8.51358595e-10]
 [ 8.08893530e-11 -3.26064465e-01  1.42419224e-10]
 [-4.03060854e-11 -3.42770743e-10 -5.05460478e-11]
 [ 1.78015783e-09  5.67179461e-02 -5.64042114e-11]
 [-1.24070597e-01  4.90545224e-11 -1.46495959e-01]
 [ 5.10216019e-11 -1.26389967e-01  9.70669584e-10]
 [-5.50038907e-02  8.39467689e-10 -8.14883711e-10]
 [-6.21017796e-01  2.02241335e-10  3.22102475e-01]
 [ 4.68690335e-01 -6.44281558e-01 -1.29518966e-09]
 [-4.97503684e-10 -1.67183227e-10 -1.46492897e-01]
 [-9.18262409e-02 -5.51600699e-10 -2.14839051e-01]
 [-8.41901879e-11 -1.45091967e-09 -2.75254279e-10]
 [ 1.16954488e-10  2.94749105e-10  1.68667788e-09]
 [ 3.58453322e-10 -5.28465053e-10  1.66821438e-10]
 [ 1.70813536e-10 -7.34411071e-10 -1.99210315e-09]
 [ 1.15488170e-10  1.20376485e-10 -5.67433700

In [53]:
MAX_ITER = 500

log_dir = 'log'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

alg = IPLUX(prob, network, alpha=1, rho=10, verbose=0)


self.prob minimize param3500 @ var3499 + norm1(var3499) + 0.5 @ QuadForm(var3499 + -param3501, [[1.00 0.00 0.00]
 [0.00 1.00 0.00]
 [0.00 0.00 1.00]]) + 0.05 @ quad_over_lin(param3502 @ var3499, 1.0) + param3503 @ var3499 + param3504[0] @ QuadForm(var3499, [[1.00 0.00 0.00]
 [0.00 1.00 0.00]
 [0.00 0.00 1.00]]) + Promote(-2.0, (3,)) @ param3505 @ var3499
subject to QuadForm(var3499 + -param3507, [[1.00 0.00 0.00]
 [0.00 1.00 0.00]
 [0.00 0.00 1.00]]) <= param3508[0]
reset
saved


In [ ]:
# alg = IPLUX(prob, network, alpha=20, rho=0.05, no_ineq=True)

# logging.info(f'{labels[algo_idx]} is running:')

np.set_printoptions(formatter={'float':lambda x: f' {x:.2e}' if x>0 else f'{x:.2e}'})

for t in range(MAX_ITER):
    alg.step()
    obj_err, cons_vio = alg.compute_metrics()
    obj_err_avg, cons_vio_avg = alg.compute_metrics(avg=True)
    
    if alg.iter_num % 100 == 0:
        logging.info(f'iter {alg.iter_num}, obj err: {obj_err:.2e}, cons vio: {cons_vio:.2e}')
# obj_err_list.append(obj_err_log)
# cons_val_list.append(cons_vio_log)






In [32]:
# =========================================================================== |
# ---------------------------------- Plot ----------------------------------- |
# =========================================================================== |


from plot_utils import MyFigure

# ================================ objective error============================
obj_err_figure = MyFigure(filename='obj_err', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel=r'$\sum_{i=1}^N f_i(\bar{x}_{i,t}) - \sum_{i=1}^N f_i(x_i^\star)$',
                            yscale='log')
obj_err_figure.add_line("IPLUX", alg.obj_err_log)
obj_err_figure.add_line("IPLUX avg", alg.obj_err_avg_log)
obj_err_figure.paint(MAX_ITER=MAX_ITER)


# ================================= constraint violation ====================
cons_vio_figure = MyFigure(filename='cons_vio', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            # ylabel=r'$\left\vert\sum_{i=1}^N g_i(\bar{x}_{i,t})\right\vert$',
                            ylabel='constraint violation',
                            yscale='log')
cons_vio_figure.add_line("IPLUX", alg.cons_vio_log)
cons_vio_figure.add_line("IPLUX avg", alg.cons_vio_avg_log)
cons_vio_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)



# ============================= trace in x1 x1 plane =========================
import matplotlib.pyplot as plt
import matplotlib.cm as cm

fig, ax = plt.subplots()     # Create a figure containing a single Axes.

delta = 0.05
x = np.arange(0.0, 1, delta)
y = np.arange(0.0, 1, delta)
X, Y = np.meshgrid(x, y)
F = X*alg.c[0] + Y*alg.c[1]
G = -alg.d[0]*np.log(1+X) - alg.d[1]*np.log(1+Y) + alg.b[0]*2/num_node

im = ax.imshow(F, interpolation='bilinear', origin='lower',
            cmap=cm.gray, extent=(0, 1, 0, 1))
CS_F = ax.contour(X, Y, F, 5)
ax.clabel(CS_F, fontsize=50)

# if there are only two nodes, we can polt the constraint
if num_node == 2:
    CS_G = ax.contour(X, Y, G, colors='k')
    ax.clabel(CS_G, fontsize=20)

x_log = np.array(alg.x_log)
x_avg_log = np.array(alg.x_avg_log)
ax.scatter(alg.x_star[0], alg.x_star[1], s=5000, c='r', marker='X', label='optimal')
ax.scatter(x_log[-1,0], x_log[-1,1], s=500, c='y', marker='^', label='last')
ax.scatter(x_log[:,0], x_log[:,1], label='iter', s=100, c='b', marker='^')  # Plot some data on the Axes.
ax.scatter(x_avg_log[:,0], x_avg_log[:,1], label='iter avg', s=100, marker='^')

ax.set_title('x position')
ax.set_xlabel('$x_0$')
ax.set_ylabel('$x_1$')
ax.legend()
fig.savefig('x.png')
plt.close()


In [7]:
print(alg.W)
print(alg.H)

[[ 6.50e-01 0.00e+00 0.00e+00 0.00e+00  1.00e-01 0.00e+00  8.33e-02
  0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00  8.33e-02 0.00e+00
  0.00e+00 0.00e+00 0.00e+00  8.33e-02 0.00e+00 0.00e+00]
 [0.00e+00  8.19e-01 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
  0.00e+00 0.00e+00 0.00e+00  1.25e-01 0.00e+00 0.00e+00 0.00e+00
   5.56e-02 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00]
 [0.00e+00 0.00e+00  7.45e-01 0.00e+00  1.00e-01 0.00e+00 0.00e+00
  0.00e+00 0.00e+00 0.00e+00 0.00e+00  7.14e-02 0.00e+00 0.00e+00
  0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00  8.33e-02]
 [0.00e+00 0.00e+00 0.00e+00  8.19e-01 0.00e+00 0.00e+00 0.00e+00
  0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
   5.56e-02 0.00e+00 0.00e+00 0.00e+00  1.25e-01 0.00e+00]
 [ 1.00e-01 0.00e+00  1.00e-01 0.00e+00  6.33e-01 0.00e+00  8.33e-02
  0.00e+00 0.00e+00  8.33e-02 0.00e+00 0.00e+00 0.00e+00 0.00e+00
  0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00]
 [0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+0